# Transform Chain 详细教程

本教程将介绍如何构建和使用Transform链进行数据处理。我们将从基础概念开始，逐步深入到实际应用。

## 第一部分：基础概念和简单示例

首先，我们需要导入必要的库并创建测试数据来演示Transform链的基本用法。

In [1]:
import torch
import numpy as np

# 创建测试数据
test_data = torch.tensor([
    [-1000., -500., 0.],
    [500., 800., 1000.]
])

print("原始数据:")
print(test_data)
print("\n数据形状:", test_data.shape)
print("数据类型:", test_data.dtype)

原始数据:
tensor([[-1000.,  -500.,     0.],
        [  500.,   800.,  1000.]])

数据形状: torch.Size([2, 3])
数据类型: torch.float32


### 3. 最简单的数据处理

在开始使用Transform链之前，让我们先看看最基本的数据处理方式。这有助于理解Transform链的优势。

 3.1 直接处理方式

In [2]:
processed_data = test_data * 2
print("直接处理的结果:")
print(processed_data)

直接处理的结果:
tensor([[-2000., -1000.,     0.],
        [ 1000.,  1600.,  2000.]])


 3.2 使用函数处理

In [3]:
def simple_process(x):
    return x * 2

processed_data = simple_process(test_data)
print("\n使用函数处理的结果:")
print(processed_data)


使用函数处理的结果:
tensor([[-2000., -1000.,     0.],
        [ 1000.,  1600.,  2000.]])


 3.3 带参数的函数处理

In [4]:
def parameterized_process(x, factor):
    return x * factor

processed_data = parameterized_process(test_data, 2)
print("\n使用带参数函数处理的结果:")
print(processed_data)


使用带参数函数处理的结果:
tensor([[-2000., -1000.,     0.],
        [ 1000.,  1600.,  2000.]])


## 第二部分：Transform构建器

接下来，我们将定义基础的TransformBuilder类，这是实现Transform链的核心组件。

In [5]:
class TransformBuilder:
    """Transform构建器：用于构建数据处理链"""
    def __init__(self):
        self.transforms = []  # 存储所有处理函数
        
    def add_custom(self, func, name: str):
        """
        添加自定义处理函数
        Args:
            func: 处理函数，必须接受一个参数并返回处理结果
            name: 函数名称，用于标识
        """
        self.transforms.append((name, func))
        return self  # 返回self支持链式调用
        
    def build(self):
        """构建最终的处理函数"""
        def transform(x):
            for name, func in self.transforms:
                x = func(x)
            return x
        return transform

### 基础示例

让我们从最简单的Transform链开始，观察它如何组合多个处理步骤。

In [6]:
# 创建简单的transform链
builder = TransformBuilder()
transform = builder\
    .add_custom(lambda x: x * 2, "double")\
    .add_custom(lambda x: x + 1, "plus_one")\
    .build()
print(test_data)
result = transform(test_data)
print("使用Transform链处理的结果:")
print(result)

tensor([[-1000.,  -500.,     0.],
        [  500.,   800.,  1000.]])
使用Transform链处理的结果:
tensor([[-1.9990e+03, -9.9900e+02,  1.0000e+00],
        [ 1.0010e+03,  1.6010e+03,  2.0010e+03]])


数学运算链示例

In [7]:
def add_value(value=1):
    """加法运算"""
    def add_func(x):
        return x + value
    return add_func

def multiply_value(value=2):
    """乘法运算"""
    def multiply_func(x):
        return x * value
    return multiply_func
# 创建测试数据
test_data = torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32)
print(test_data)

tensor([1., 2., 3., 4., 5.])


构建数学运算链

In [8]:
math_transform = TransformBuilder()\
    .add_custom(add_value(10), "加10")\
    .add_custom(multiply_value(2), "乘2")\
    .add_custom(add_value(1), "加3")\
    .build()

result = math_transform(test_data)
print("数学运算结果:")
print(f"原始数据: {test_data}")
print(f"处理后: {result}")

数学运算结果:
原始数据: tensor([1., 2., 3., 4., 5.])
处理后: tensor([23., 25., 27., 29., 31.])


简单数据处理链

In [9]:
def round_numbers(decimals=0):
    """四舍五入"""
    def round_func(x):
        return torch.round(x * (10 ** decimals)) / (10 ** decimals)
    return round_func

def set_range(min_val=0, max_val=10):
    """限定数值范围"""
    def range_func(x):
        return torch.clamp(x, min_val, max_val)
    return range_func

# 创建测试数据
test_data = torch.tensor([1.23, 5.67, 8.91, 12.34, -3.45])
print(test_data)

tensor([ 1.2300,  5.6700,  8.9100, 12.3400, -3.4500])


构建数据处理链

In [10]:
process_transform = TransformBuilder()\
    .add_custom(round_numbers(1), "保留1位小数")\
    .add_custom(set_range(0, 10), "限制范围0-10")\
    .build()

result = process_transform(test_data)
print("\n数据处理结果:")
print(f"原始数据: {test_data}")
print(f"处理后: {result}")


数据处理结果:
原始数据: tensor([ 1.2300,  5.6700,  8.9100, 12.3400, -3.4500])
处理后: tensor([ 1.2000,  5.7000,  8.9000, 10.0000,  0.0000])


实用工具函数

In [11]:
def print_steps(transform_chain, data):
    """打印每一步的处理结果"""
    print("\n逐步处理过程:")
    x = data
    print(f"输入数据: {x}")
    
    for name, func in transform_chain.transforms:
        x = func(x)
        print(f"{name}: {x}")
    
    return x

创建一个简单的处理链

In [12]:
demo_builder = TransformBuilder()\
    .add_custom(add_value(5), "加5")\
    .add_custom(multiply_value(2), "乘2")\
    .add_custom(round_numbers(1), "保留1位小数")

测试数据

In [13]:
demo_data = torch.tensor([1.2, 3.4, 5.6])
print(demo_data)

tensor([1.2000, 3.4000, 5.6000])


查看处理过程

In [14]:
print_steps(demo_builder, demo_data)


逐步处理过程:
输入数据: tensor([1.2000, 3.4000, 5.6000])
加5: tensor([ 6.2000,  8.4000, 10.6000])
乘2: tensor([12.4000, 16.8000, 21.2000])
保留1位小数: tensor([12.4000, 16.8000, 21.2000])


tensor([12.4000, 16.8000, 21.2000])

如果需要最终的转换函数，可以在这里构建

In [15]:
demo_transform = demo_builder.build()
print(demo_transform)

<function TransformBuilder.build.<locals>.transform at 0x000001B71FCE0940>


## 第三部分：实际应用

在实际应用中，我们需要一些更实用的处理函数。这些函数通常需要参数来控制处理效果。

In [16]:
def normalize(min_val=-1000.0, max_val=1000.0):
    """归一化函数：将数据映射到[-1,1]范围"""
    def normalize_func(x):
        return (x - min_val) / (max_val - min_val) * 2 - 1
    return normalize_func

def window(center=40, width=400):
    """窗位窗宽处理：用于突出特定范围的数据"""
    def window_func(x):
        min_val = center - width/2
        max_val = center + width/2
        return torch.clamp(x, min_val, max_val)
    return window_func

def enhance_contrast(factor=1.5):
    """对比度增强：通过拉伸数据范围来增强对比度"""
    def contrast_func(x):
        mean = x.mean()
        return torch.clamp((x - mean) * factor + mean, -1, 1)
    return contrast_func

### 7. 创建完整的处理链

现在我们可以组合这些处理函数，创建一个完整的处理流程。

In [17]:
# 7.1 基础处理链
transform_basic = TransformBuilder()\
    .add_custom(normalize(), "normalize")\
    .add_custom(window(), "window")\
    .build()

result_basic = transform_basic(test_data)
print("基础处理链结果:")
print(result_basic)

# 7.2 添加对比度增强
transform_enhanced = TransformBuilder()\
    .add_custom(normalize(), "normalize")\
    .add_custom(window(), "window")\
    .add_custom(enhance_contrast(), "contrast")\
    .build()

result_enhanced = transform_enhanced(test_data)
print("\n增强处理链结果:")
print(result_enhanced)

基础处理链结果:
tensor([ 0.0012,  0.0057,  0.0089,  0.0123, -0.0035])

增强处理链结果:
tensor([-0.0006,  0.0060,  0.0109,  0.0160, -0.0076])


## 第四部分：进阶应用

在实际应用中，我们可能需要根据不同情况动态选择使用哪些处理步骤。

In [18]:
def build_dynamic_transform(need_normalize=True, 
                          need_window=True,
                          need_contrast=False,
                          contrast_factor=1.5):
    """动态构建处理链：根据参数决定使用哪些处理步骤"""
    builder = TransformBuilder()
    
    if need_normalize:
        builder.add_custom(normalize(), "normalize")
    
    if need_window:
        builder.add_custom(window(), "window")
        
    if need_contrast:
        builder.add_custom(enhance_contrast(contrast_factor), "contrast")
        
    return builder.build()

# 测试不同组合
transforms = {
    "仅归一化": build_dynamic_transform(need_window=False, need_contrast=False),
    "归一化+窗口": build_dynamic_transform(need_contrast=False),
    "完整处理": build_dynamic_transform(need_contrast=True),
}

for name, transform in transforms.items():
    result = transform(test_data)
    print(f"\n{name}结果:")
    print(result)


仅归一化结果:
tensor([ 0.0012,  0.0057,  0.0089,  0.0123, -0.0035])

归一化+窗口结果:
tensor([ 0.0012,  0.0057,  0.0089,  0.0123, -0.0035])

完整处理结果:
tensor([-0.0006,  0.0060,  0.0109,  0.0160, -0.0076])


## 第五部分：实践练习

### 9. 练习：创建自定义处理函数

尝试创建你自己的处理函数并添加到处理链中。注意以下要点：
1. 函数应该只接受一个输入参数
2. 函数应该返回处理后的结果
3. 注意数据类型的一致性

In [19]:
def custom_process(x):
    # 对数变换示例
    return torch.log(torch.abs(x) + 1) * torch.sign(x)

# 将自定义函数添加到处理链
transform_custom = TransformBuilder()\
    .add_custom(normalize(), "normalize")\
    .add_custom(custom_process, "custom")\
    .build()

result_custom = transform_custom(test_data)
print("自定义处理结果:")
print(result_custom)

自定义处理结果:
tensor([ 0.0012,  0.0057,  0.0089,  0.0123, -0.0034])


### 10. 完整示例：处理CT数据

这个示例展示了如何处理实际的CT数据，包括完整的预处理流程。

In [20]:
def process_ct_data(data, window_center=40, window_width=400, contrast_factor=1.5):
    """完整的CT数据处理流程"""
    transform = TransformBuilder()\
        .add_custom(normalize(), "normalize")\
        .add_custom(window(window_center, window_width), "window")\
        .add_custom(enhance_contrast(contrast_factor), "contrast")\
        .build()
        
    return transform(data)

# 使用示例
processed_ct = process_ct_data(test_data)
print("CT数据处理结果:")
print(processed_ct)

CT数据处理结果:
tensor([-0.0006,  0.0060,  0.0109,  0.0160, -0.0076])


## 总结

本教程介绍了Transform链的基本概念和用法。我们还展示了如何构建Transform链，并使用了一些实际应用中的例子。最后，我们还介绍了如何动态构建Transform链，并展示了如何处理实际的CT数据。

Transform链是一个强大的工具，可以帮助我们处理复杂的数据。它可以帮助我们对数据进行预处理、数据增强、数据转换等操作，从而提升模型的性能。